In [5]:
#Modules to install via pip pandas,ipynb
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import json
from pprint import pprint
import os
import import_ipynb
import sys

sys.path.append('../')
from functions import *
from trace_analysis import *
from pandas.plotting import scatter_matrix
from trace_analysis_cooja2 import *


In [7]:

#+"/cooja-9nodes/"
#print(directory)

plots = [("traces/normal", 'grid9_normal_2019-02-11_17:51:17_'),
         ("traces/normal", 'grid9_normal_2019-02-11_20:22:01_'),
         ("traces/1bh-6", 'grid9_1bh-6_2019-02-11_20:48:08_'),
         ("traces/1bh-6", 'grid9_1bh-6_2019-02-11_21:03:19_')]

for row in plots:
    nodes, packets_node = process_cooja2_traces(row[0], row[1])


In [43]:
def process_cooja2_traces(path, tracemask):
    files = []

    # load all files and extract IPs of nodes
    for file in os.listdir(path):
        try:
            if file.startswith(tracemask) and file.index("routes"):
                continue
        except:
            files.append(file)

    nodes = pd.DataFrame(columns=['node_id', 'rank'])
    packets_node = {}

    # Load the ICMP traces
    for file in files:
        packets = pd.read_csv(path + '/' + file,
                              sep=' |icmp_seq=|ttl=|time=',
                              na_filter=True,
                              header=None,
                              skiprows=1,
                              skipfooter=4,
                              usecols=[3, 5, 7, 9],
                              names=['node_id', 'seq', 'hop', 'rtt'],
                              engine='python').dropna().drop_duplicates()

        nodes.loc[len(nodes)] = [packets['node_id'][0], 64 - packets['hop'][0]]

        packets = packets.sort_values(by=['node_id', 'seq'], ascending=True, na_position='first')
        packets = packets[packets['rtt'] > 1]

        packets_node[packets['node_id'][0]] = packets

    return nodes.sort_values(by=['rank','node_id']), packets_node





In [48]:
directory=os.getcwd()
traces=directory+"/traces"
traces+="/normal"
print(traces)
tracemask="grid9_normal_2019-02-11_17:51:17_"

def import_Cooja_2(directory,tracemask):
    print(directory)
    print(tracemask)
    files = []
    data[]
    # load all files and extract IPs of nodes
    for file in os.listdir(directory):
        try:
            if file.startswith(tracemask) and file.index("routes"):
                continue
        except:
            files.append(file)
    for file in files:
        packets = pd.read_csv(directory + '/' + file,
                              sep=' |icmp_seq=|ttl=|time=',
                              na_filter=True,
                              header=None,
                              skiprows=1,
                              skipfooter=4,
                              usecols=[3, 5, 7, 9],
                              names=['node_id', 'seq', 'hop', 'rtt'],
                              engine='python').dropna().drop_duplicates()

        print(packets.head())
        
        #nodes.loc[len(nodes)] = [packets['node_id'][0], 64 - packets['hop'][0]]

        #packets = packets.sort_values(by=['node_id', 'seq'], ascending=True, na_position='first')
        #packets = packets[packets['rtt'] > 1]

        #packets_node[packets['node_id'][0]] = packets
        
    print(type(packets))
    return nodes,packets_node

/home/fedebyes/Workspace/Master Thesis/iot-netprofiler/cooja2-9nodes/traces/normal


In [49]:

data,packets_node=import_Cooja_2(traces,tracemask)
packets_node

/home/fedebyes/Workspace/Master Thesis/iot-netprofiler/cooja2-9nodes/traces/normal
grid9_normal_2019-02-11_17:51:17_
                 node_id  seq  hop   rtt
0  fd00::212:7409:9:909:    2   61   858
1  fd00::212:7409:9:909:    3   61   513
2  fd00::212:7409:9:909:    4   61   287
3  fd00::212:7409:9:909:    5   61   223
4  fd00::212:7409:9:909:    6   61  1054
                 node_id  seq  hop    rtt
0  fd00::212:7404:4:404:    1   62  106.0
1  fd00::212:7404:4:404:    3   62  876.0
2  fd00::212:7404:4:404:    4   62  206.0
3  fd00::212:7404:4:404:    5   62  170.0
4  fd00::212:7404:4:404:    6   62  207.0
                 node_id  seq  hop    rtt
0  fd00::212:7403:3:303:    1   63   61.0
1  fd00::212:7403:3:303:    3   63   72.5
2  fd00::212:7403:3:303:    6   63   59.4
3  fd00::212:7403:3:303:   11   63  209.0
4  fd00::212:7403:3:303:   12   63  707.0
                 node_id  seq  hop  rtt
0  fd00::212:7405:5:505:    1   61  482
1  fd00::212:7405:5:505:    3   61  239
2  fd00::212:

'rank'